In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(640, 640, 3))

x = GlobalAveragePooling2D()(base_model_resnet.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model_resnet = Model(inputs=base_model_resnet.input, outputs=predictions)

for layer in base_model_resnet.layers:
    layer.trainable = False

model_resnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

94765736/94765736 [==============================] - 1s 0us/step


In [3]:
from tensorflow.keras.applications import InceptionV3

base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(640, 640, 3))

x = GlobalAveragePooling2D()(base_model_inception.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model_inception = Model(inputs=base_model_inception.input, outputs=predictions)

for layer in base_model_inception.layers:
    layer.trainable = False

model_inception.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
!apt-get install unrar
!unrar x "/content/drive/MyDrive/Fall Detection.rar" "/content/Fall Detection/"

In [21]:
%cd /content/Fall Detection

/content/Fall Detection


In [25]:
import pandas as pd

train_annotations = pd.read_csv("Fall Detection/train/_annotations.csv")
valid_annotations = pd.read_csv("Fall Detection/valid/_annotations.csv")
test_annotations = pd.read_csv("Fall Detection/test/_annotations.csv")

class_map = {'Fall-Detected': 1, 'Not-Fall-Detected': 0}
train_annotations['class'] = train_annotations['class'].astype(str)
valid_annotations['class'] = valid_annotations['class'].astype(str)
test_annotations['class'] = test_annotations['class'].astype(str)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_annotations,
    directory='Fall Detection/train/',
    x_col="filename",
    y_col="class",
    target_size=(640, 640),
    batch_size=32,
    class_mode='binary')

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_annotations,
    directory='Fall Detection/valid/',
    x_col="filename",
    y_col="class",
    target_size=(640, 640),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_annotations,
    directory='Fall Detection/test/',
    x_col="filename",
    y_col="class",
    target_size=(640, 640),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 9442 validated image filenames belonging to 2 classes.
Found 900 validated image filenames belonging to 2 classes.
Found 451 validated image filenames belonging to 2 classes.


In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_resnet = ModelCheckpoint('/content/drive/MyDrive/best_model_resnet.h5', save_best_only=True, monitor='val_loss', verbose=1)

history_resnet = model_resnet.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=50,
    callbacks=[checkpoint_resnet]
)

checkpoint_inception = ModelCheckpoint('/content/drive/MyDrive/best_model_inception.h5', save_best_only=True, monitor='val_loss', verbose=1)

history_inception = model_inception.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=50,
    callbacks=[checkpoint_inception]
)

Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 0.0090 - accuracy: 0.9900


Epoch 1: val_loss improved from inf to 0.05318, saving model to /content/drive/MyDrive/best_model_inception.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 372s 3s/step - loss: 0.0090 - accuracy: 0.9900 - val_loss: 0.0532 - val_accuracy: 0.9989
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 5.6898e-15 - accuracy: 1.0000

100/100 [==============================] - 331s 3s/step - loss: 5.6898e-15 - accuracy: 1.0000
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 2.2121e-15 - accuracy: 1.0000

...

100/100 [==============================] - 328s 3s/step - loss: 0.0058 - accuracy: 0.9997
Epoch 7/10
100/100 [==============================] - ETA: 0s - loss: 4.4902e-09 - accuracy: 1.0000

100/100 [==============================] - 326s 3s/step - loss: 4.4902e-09 - accuracy: 1.0000
Epoch 8/10
100/100 [==============================] - ETA: 0s - loss: 3.1391e-09 - accuracy: 1.0000

100/100 [==============================] - 326s 3s/step - loss: 3.1391e-09 - accuracy: 1.0000
Epoch 9/10
100/100 [==============================] - ETA: 0s - loss: 2.7116e-09 - accuracy: 1.0000

100/100 [==============================] - 324s 3s/step - loss: 2.7116e-09 - accuracy: 1.0000
Epoch 10/10
100/100 [==============================] - ETA: 0s - loss: 2.4644e-09 - accuracy: 1.0000

100/100 [==============================] - 323s 3s/step - loss: 2.4644e-09 - accuracy: 1.0000


In [24]:
from tensorflow.keras.models import load_model

model_resnet = load_model('/content/drive/MyDrive/best_model_resnet.h5')
model_inception = load_model('/content/drive/MyDrive/best_model_inception.h5')

In [26]:
test_loss_resnet, test_accuracy_resnet = model_resnet.evaluate(test_generator)
print(f"ResNet Test Accuracy: {test_accuracy_resnet}")

test_loss_inception, test_accuracy_inception = model_inception.evaluate(test_generator)
print(f"Inception Test Accuracy: {test_accuracy_inception}")

15/15 [==============================] - 23s 798ms/step - loss: 0.1906 - accuracy: 0.9978
ResNet Test Accuracy: 0.9977827072143555
15/15 [==============================] - 13s 786ms/step - loss: 0.1061 - accuracy: 0.9978
Inception Test Accuracy: 0.9977827072143555


In [27]:
val_loss_resnet, val_accuracy_resnet = model_resnet.evaluate(valid_generator)
print(f"ResNet Validation Accuracy: {val_accuracy_resnet}")

val_loss_inception, val_accuracy_inception = model_inception.evaluate(valid_generator)
print(f"Inception Validation Accuracy: {val_accuracy_inception}")

29/29 [==============================] - 24s 813ms/step - loss: 0.0955 - accuracy: 0.9989
ResNet Validation Accuracy: 0.9988889098167419
29/29 [==============================] - 22s 739ms/step - loss: 0.0532 - accuracy: 0.9989
Inception Validation Accuracy: 0.9988889098167419


In [28]:
from sklearn.metrics import classification_report, confusion_matrix

predictions_resnet = model_resnet.predict(test_generator)
predictions_resnet = [1 if x > 0.5 else 0 for x in predictions_resnet]

print(classification_report(test_generator.classes, predictions_resnet))
print(confusion_matrix(test_generator.classes, predictions_resnet))

15/15 [==============================] - 12s 725ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       450
           1       0.00      0.00      0.00         1

    accuracy                           1.00       451
   macro avg       0.50      0.50      0.50       451
weighted avg       1.00      1.00      1.00       451

[[450   0]
 [  1   0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
